<a href="https://colab.research.google.com/github/dolDolSee/TF2.0/blob/main/tensorflowPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy

W = tf.Variable(tf.random.normal(shape=[1]))
b = tf.Variable(tf.random.normal(shape=[1]))

#모델정의
@tf.function
def linear_model(x):
  return W*x + b

#loss함수 정의
@tf.function
def mse_loss(y_pred, y):
  return tf.reduce_mean(tf.square(y_pred - y))

#optimizer정의
optimizer = tf.optimizers.SGD(0.01)

#최적화를 위한 함수 정의
@tf.function
def train_step(x,y):
  with tf.GradientTape() as tape:
    y_pred = linear_model(x)
    loss = mse_loss(y_pred, y)
  gradients = tape.gradient(loss, [W,b])
  optimizer.apply_gradients(zip(gradients, [W,b])) 

x_train = [1,2,3,4]
y_train = [2,4,6,8]

for i in range(1000):
  train_step(x_train, y_train)

x_test = [3.5,5,5.5,6]

print(linear_model(x_test).numpy())

[ 6.993254  9.975182 10.969157 11.963133]


### mnist 분류기 구현

In [1]:
import tensorflow as tf

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
#딥러닝에서는 실수형을 사용하기 때문에 int형에서 변경해줌
x_train, x_test = x_train.astype("float32"), x_test.astype("float32")
#flatting: 2차원 메트릭스를 1차원 벡터의 784(28*28)으로 변환해줌
x_train, x_test = x_train.reshape([-1,784]), x_test.reshape([-1,784])
x_train, x_test = x_train/255., x_test /255.

y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)



11501568/11490434 [==============================] - 0s 0us/step


In [2]:
#데이터 미니배치 단위로 묶기
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#batch함수로 원하는 미니배치 갯수 지정 : 60000개의 mnist데이터를 100개 단위로 묶겠다., 다 묶으면 shuffle함
train_data = train_data.repeat().shuffle(60000).batch(100)
#배치를 순회해서 가져옴
train_data_iter = iter(train_data)

In [6]:
#가설정의
class SoftmaxRegression(tf.keras.Model):
  def __init__(self):
    super(SoftmaxRegression, self).__init__()
    self.softmax_layer = tf.keras.layers.Dense(10,
                                               activation = None,
                                               #W값
                                               kernel_initializer = "zeros",
                                               #B값
                                               bias_initializer="zeros")
  def call(self, x):
    #인자를 넣어서 softmax_layer연산 수행
    logits = self.softmax_layer(x)
    #Dense를 지난 결과에 softmax 씌워서 반환
    return tf.nn.softmax(logits)

#손실함수 정의
@tf.function
def cross_entropy_loss(y_pred, y):
  return tf.reduce_mean(-tf.reduce_sum(y*tf.math.log(y_pred),axis=[1]))

#Optimizer
optimizer = tf.optimizers.SGD(0.5)

#GD정의
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = cross_entropy_loss(y_pred, y)
  #GD가 계산되는 시점, model.trainable_variable은 자동으로 갱신이 요구되는 변수값을 할당해줌.
  gradients = tape.gradient(loss, model.trainable_variables)
  #Optimizer를 통해서 갱신이 이루어지는 시점
  optimizer.apply_gradients(zip(gradients,model.trainable_variables))

#정확도 출력 결과 정의
@tf.function
def compute_accuracy(y_pred, y):
  #연산 결과는 10개의 요소를 가진 1차원 벡터이므로 결과 비교를 위해서 argmax를 통해 10개중 가장 큰 값을 뽑아서 equal로 비교함.
  corret_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(corret_prediction, tf.float32))
  
  return accuracy


SoftmaxRegression_model = SoftmaxRegression()

for i in range(1000):
  batch_xs, batch_ys = next(train_data_iter)
  train_step(SoftmaxRegression_model, batch_xs, batch_ys)

print(compute_accuracy(SoftmaxRegression_model(x_test), y_test))

  


tf.Tensor(0.9175, shape=(), dtype=float32)


In [11]:
import tensorflow as tf

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
#딥러닝에서는 실수형을 사용하기 때문에 int형에서 변경해줌
x_train, x_test = x_train.astype("float32"), x_test.astype("float32")
#flatting: 2차원 메트릭스를 1차원 벡터의 784(28*28)으로 변환해줌
x_train, x_test = x_train.reshape([-1,784]), x_test.reshape([-1,784])
x_train, x_test = x_train/255., x_test /255.

y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)

In [19]:
learning_rate = 0.001
num_epochs = 30
batch_size = 256
display_step = 1 #손실함수추출력
input_size = 784
hidden1_size = 256
hidden2_size = 256
output_size = 10

In [20]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(60000).batch(batch_size)

In [ ]:
#초기 가중치 값을 초기화하는 함수
def random_normal_initializer_with_stddev_1():
  return tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.0, seed=None)

class ANN(tf.keras.Model):
  def __init__(self):
    super(ANN, self).__init__()
    self.hidden_layer_1 = tf.keras.layers.Dense(hidden1_size,
                                                activation="relu",
                                                kernel_initializer=random_normal_initializer_with_stddev_1(),
                                                bias_initializer=random_normal_initializer_with_stddev_1())
    self.hidden_layer_2 = tf.keras.layers.Dense(hidden2_size,
                                         activation="relu",
                                         kernel_initializer=random_normal_initializer_with_stddev_1(),
                                         bias_initializer=random_normal_initializer_with_stddev_1())
    self.output_layer = tf.keras.layers.Dense(output_size,
                                       activation=None,
                                       kernel_initializer=random_normal_initializer_with_stddev_1(),
                                       bias_initializer=random_normal_initializer_with_stddev_1())
    
  def call(self, x):
    H1_output = self.hidden_layer_1(x)
    H2_output = self.hidden_layer_2(H1_output)
    logits = self.output_layer(H2_output)
    return logits

#손실함수 정의
@tf.function
def cross_entropy_loss(logits, y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

#optimizer
optimizer = tf.optimizers.Adam(learning_rate)

#최적화 function
@tf.function
def train_step(model, x,y):
  with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = cross_entropy_loss(y_pred, y)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

#정확도 출력 함수
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  return accuracy

ANN_model  = ANN()

for epoch in range(num_epochs):
  average_loss =0.
  total_batch = int(x_train.shape[0]/batch_size)

  for batch_x, batch_y in train_data:
    _, current_loss = train_step(ANN_model, batch_x, batch_y), cross_entropy_loss(ANN_model(batch_x), batch_y)
    average_loss +=current_loss/total_batch
  if epoch % display_step ==0:
    print("반복(Epoch): %d, 손실함수(Loss):%f" % ((epoch+1), average_loss))

print(compute_accuracy(ANN_model(x_test),y_test))

